# 层次化索引

In [1]:
import numpy as np
import pandas as pd
from numpy import nan as NA
from pandas import DataFrame
from pandas import MultiIndex
from pandas import Series

In [2]:
data = Series(np.random.randn(10),
              index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
              [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

# cc： 这里可以看到index传入的是二维列表

a  1   -1.383210
   2    2.583529
   3   -0.187641
b  1    1.422343
   2   -0.808011
   3    1.266836
c  1   -1.913419
   2    0.877287
d  2    0.701944
   3   -1.288192
dtype: float64

In [4]:
data.index # labels是levels的对应展开


MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
data['b']


1    0.868490
2    0.460032
3   -1.856422
dtype: float64

In [6]:
data['b': 'c'] # 闭区间


b  1    0.868490
   2    0.460032
   3   -1.856422
c  1    0.357513
   2   -0.838404
dtype: float64

In [7]:
data.loc[['a', 'd']] # 选取索引a和d


a  1   -1.077088
   2   -1.253854
   3    0.056602
d  2    0.718121
   3    1.343245
dtype: float64

## df.stack() 和 df.unstack()

In [8]:
data.unstack() # 把第二级行索引变成列，因为我们现在是Series一维数组。

# cc：直译，不堆叠，反之stack()则为堆叠

,1,2,3
a,-1.077088,-1.253854,0.056602
b,0.868490,0.460032,-1.856422
c,0.357513,-0.838404,NaN
d,NaN,0.718121,1.343245


In [10]:
# cc：unstack()之后变成了 DataFrame类型
type(data.unstack())

pandas.core.frame.DataFrame

In [11]:
data.unstack().stack() # 再叠回来


a  1   -1.077088
   2   -1.253854
   3    0.056602
b  1    0.868490
   2    0.460032
   3   -1.856422
c  1    0.357513
   2   -0.838404
d  2    0.718121
   3    1.343245
dtype: float64

In [12]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [14]:
frame['Ohio'] # 直接选取列


color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

## 构造多级索引 - MultiIndex

In [15]:
mi = MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'],
                             ['Green', 'Red', 'Green']], 
                            names=['state', 'color'])
print(mi)

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])


In [16]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]])
frame.columns = mi # 直接设置MultiIndex对象
frame

state  Ohio     Colorado
color Green Red    Green
a 1       0   1        2
  2       3   4        5
b 1       6   7        8
  2       9  10       11

## 重排分级顺序

In [20]:
# 重置数据
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

## 交换索引顺序 - df.swaplevel()


In [19]:
frame.swaplevel('key1', 'key2') # 交换索引顺序


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [26]:
frame.sort_index(level=1) # 对第一层索引排序，不再用sortlevel函数。


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [22]:
frame.swaplevel(0, 1).sort_index(level=0)


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [27]:
# 根据级别汇总统计
frame.sum(level='key2') # 根据key2的索引求和，每一列上对应行索引相同的元素相加求和。

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [28]:
# level对应行还是列由axis决定。
# 每一列上对应列相同的元素相加求和。
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [29]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                   'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


## 把列转为索引 - df.set_index()

In [30]:
# 把列转为索引
frame2 = frame.set_index(['c', 'd']) # 默认drop为True
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [31]:
# cc，如果把drop 改为False 会发现依然会保留c和d列，实际上这种情况用的少，一般还是按照默认值操作即可~
frame3 = frame.set_index(['c', 'd'], drop=False) # 默认drop为True
frame3

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [32]:
frame2 = frame.set_index(['c', 'd']) # 默认drop为True
frame2
frame2.reset_index() # 把索引再变回列，如果之前drop为False，会因为列同名产生冲突。cc：这点注意下~

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
